## Tratar os dados do TSE dos candidatos

### fonte: https://dadosabertos.tse.jus.br/dataset/?groups=candidatos

* Usei os dados a partir de 1988 até 2020.

In [2]:
import pandas as pd
import glob

In [8]:
# Arquivos csv originais do TSE nos anos das eleições municipais são muito pesados para meu note roda em um laço
# Vamos separar os anos para facilitar na leitura também, vou dividir em eleições municipais e federais
# Anos federais 1998, 2002, 2006, 2010, 2014, 2018

colunas_para_o_projeto_federal = ["ANO_ELEICAO", "NR_TURNO", "SG_UF", "DS_CARGO", "NM_CANDIDATO", "SG_PARTIDO", "NM_PARTIDO",
                            "DT_NASCIMENTO", "DS_GENERO", "DS_GRAU_INSTRUCAO", "DS_COR_RACA", "DS_SIT_TOT_TURNO"]


df = pd.read_csv(r"/home/sposigor/Downloads/TSE/candidatos/dados_nao_tratados/2018.csv", sep=";", encoding="ISO-8859-1", low_memory=False, usecols = colunas_para_o_projeto_federal)

df.to_csv("/home/sposigor/Downloads/TSE/candidatos/dados_tratados/2018.csv", sep=";", index=False)

In [20]:
# Anos municipias 2000, 2004, 2008, 2012, 2016, 2020
# A diferença é somente a coluna de de NM_UE que indica o municipio

colunas_para_o_projeto_municipal = ["ANO_ELEICAO", "NR_TURNO", "SG_UF", "NM_UE", "DS_CARGO", "NM_CANDIDATO", "SG_PARTIDO", "NM_PARTIDO",
                            "DT_NASCIMENTO", "DS_GENERO", "DS_GRAU_INSTRUCAO", "DS_COR_RACA", "DS_SIT_TOT_TURNO"]

df = pd.read_csv(r"/home/sposigor/Downloads/TSE/candidatos/dados_nao_tratados/2020.csv", sep=";", encoding="ISO-8859-1", low_memory=False, usecols = colunas_para_o_projeto_municipal)


df.to_csv("/home/sposigor/Downloads/TSE/candidatos/dados_tratados/2020.csv", sep=";", index=False)

In [4]:
# Vamos mesclar os csv em um unico para trabalhar com ele

path = r"/home/sposigor/Downloads/TSE/candidatos/dados_tratados/federal"
todos_arquivos = glob.glob(path + "/*.csv")

arquivos = []

for arq in todos_arquivos:
        df = pd.read_csv(arq, sep=";")
        arquivos.append(df)

arquivo_completo = pd.concat(arquivos, axis=0, ignore_index=True)

arquivo_completo.to_csv("/home/sposigor/Downloads/TSE/resultado/dados_tratados/consolidado_federal.csv", sep=";", index=False)

In [ ]:
# Vamos mesclar os csv em um unico para trabalhar com ele

path = r"/home/sposigor/Downloads/TSE/candidatos/dados_tratados/municipal"
todos_arquivos = glob.glob(path + "/*.csv")

arquivos = []

for arq in todos_arquivos:
        df = pd.read_csv(arq, sep=";")
        arquivos.append(df)

arquivo_completo = pd.concat(arquivos, axis=0, ignore_index=True)

arquivo_completo.to_csv("/home/sposigor/Downloads/TSE/candidatos/dados_tratados/consolidado_municipal.csv", sep=";", index=False)

In [32]:
# Vamos ajustar alguns informações para facilitar o processo de visualização dos dados

# Iniciando com o nivel federal
df = pd.read_csv(r"/home/sposigor/Downloads/TSE/candidatos/dados_tratados/consolidado_federal.csv", sep=";", encoding="ISO-8859-1", low_memory=False)

# A coluna referente a raça
lista_mudar_descricao = ["#NE#", "#NE", "NAO DIVULGAVEL"]

for x in lista_mudar_descricao:
    df.loc[df["DS_COR_RACA"] == x, "DS_COR_RACA"] = "NAO INFORMADO"


# A coluna referente aos cargos, ajustar os cargos que são diretamente eleitos
lista_drop_cargo = ["1Âº SUPLENTE SENADOR", "2Âº SUPLENTE SENADOR", "2Âº SUPLENTE", "1Âº SUPLENTE", "VICE-PRESIDENTE", "VICE-GOVERNADOR"]
for x in lista_drop_cargo:
    drop_cargos = df[df["DS_CARGO"] == x].index
    df.drop(drop_cargos, inplace=True)
df.loc[df["DS_CARGO"] == "DEPUTADO DISTRITAL", "DS_CARGO"] = "DEPUTADO ESTADUAL" # Deputado distrital é o deputado estadual do DF


# A coluna referente a situação durante a incrisção e periodo pre e pos eleitoral

lista_drop_situação = ["#NULO#", "RENUNCIA/FALECIMENTO/CASSACAO ANTES DA ELEICAO", "REGISTRO NEGADO ANTES DA ELEICAO", "RENUNCIA/FALECIMENTO/CASSACAO APOS A ELEICAO",
                        "#NULO", "INDEFERIDO COM RECURSO", "CASSADO COM RECURSO", "SUBSTITUIDO", "REGISTRO NEGADO APOS A ELEICAO", "2Âº TURNO"]

for x in lista_drop_situação:
    drop_situacao = df[df["DS_SIT_TOT_TURNO"] == x].index
    df.drop(drop_situacao, inplace=True)


# Algumas datas de nascimento (quantidade 11), estão sem informação vou ajustar para 01/01/1980
df["DT_NASCIMENTO"].fillna("01/01/1980", inplace=True)

df.to_csv("/home/sposigor/Downloads/TSE/candidatos/dados_tratados/federal_final.csv", sep=";", index=False)

In [3]:
# Vamos ajustar alguns informações para facilitar o processo de visualização dos dados

# Iniciando com o nivel municipal
df = pd.read_csv(r"/home/sposigor/Downloads/TSE/candidatos/dados_tratados/consolidado_municipal.csv", sep=";", low_memory=True)

# A coluna referente a raça
lista_mudar_descricao = ["#NE#", "#NE", "NAO DIVULGAVEL"]

for x in lista_mudar_descricao:
    df.loc[df["DS_COR_RACA"] == x, "DS_COR_RACA"] = "NAO INFORMADO"


# A coluna referente aos cargos, ajustar os cargos que são diretamente eleitos
drop_cargos = df[df["DS_CARGO"] == "VICE-PREFEITO"].index
df.drop(drop_cargos, inplace=True)


# A coluna referente a situação durante a incrisção e periodo pre e pos eleitoral
lista_drop_situação = ['REGISTRO NEGADO ANTES DA ELEIÇAO', 'REGISTRO NEGADO APOS A ELEIÇAO', 'RENUNCIA/FALECIMENTO/CASSAÇAO ANTES DA ELEIÇAO', 
                        'RENUNCIA/FALECIMENTO/CASSAÇAO APOS A ELEIÇAO', 'RENUNCIA/FALECIMENTO COM SUBSTITUIÇAO', "#NULO#", "INDEFERIDO COM RECURSO", "SUBSTITUIDO", "2º TURNO"]

for x in lista_drop_situação:
    drop_situacao = df[df["DS_SIT_TOT_TURNO"] == x].index
    df.drop(drop_situacao, inplace=True)


# Algumas datas de nascimento (quantidade 678), estão sem informação vou ajustar para 01/01/1980
df["DT_NASCIMENTO"].fillna("01/01/1980", inplace=True)

df.to_csv("/home/sposigor/Downloads/TSE/candidatos/dados_tratados/municipal_final.csv", sep=";", index=False)